## Классификация текстов: спам-фильтр для SMS

В этом задании мы возьмем открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построим на нем классификатор текстов на эти два класса, оценим его качество с помощью кросс-валидации, протестируем его работу на отдельных примерах, и посмотрим, что будет происходить с качеством, если менять параметры модели.

In [82]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

Загрузим датасет и подготовим два списка:
* список текстов в порядке их следования в датасете
* список соответствующих им меток классов.

В качестве метки класса используем 1 для спама и 0 для "не спама".

In [20]:
with open('SMSSpamCollection.txt', 'r', encoding='utf-8') as file:
    sms = file.read().splitlines()

In [28]:
sms[:5]

['ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'ham\tOk lar... Joking wif u oni...',
 "spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'ham\tU dun say so early hor... U c already then say...',
 "ham\tNah I don't think he goes to usf, he lives around here though"]

In [49]:
sms_labels, sms_text = zip(*[line.split('\t') for line in sms])

In [74]:
labels = np.array([0 if word == 'ham' else 1 for word in sms_labels])
labels

array([0, 0, 1, ..., 0, 0, 0])

Создадим из списка текстов матрицу признаков Х и оценим качество классификации текстов с помощью LogisticRegression().

In [71]:
clf_pipeline = Pipeline(
            [('vectorizer', CountVectorizer()),
            ('regression', LogisticRegression())])


score = cross_val_score(clf_pipeline, sms_text, labels, scoring='f1', cv=10).mean()

In [72]:
print('Cross_val_score f1: %.4f' % score)

Cross_val_score f1: 0.9311


Теперь обучим классификатор на всей выборке и спрогнозируем с его помощью класс для следующих сообщений:

In [79]:
test_text = ['FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB',
            'FreeMsg: Txt: claim your reward of 3 hours talk time',
            'Have you visited the last lecture on physics?',
            'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$',
            'Only 99$']

In [75]:
clf_pipeline.fit(sms_text, labels)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('regression', LogisticRegression())])

In [78]:
clf_pipeline.predict(test_text)

array([1, 1, 0, 0, 0])

Зададим в CountVectorizer три разных параметра ngram_range. Во всех случаях измерим получившееся в кросс-валидации значение f1-меры. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы.

In [86]:
ngram_range = [(2, 2), (3, 3), (1, 3)]
scores = []
for n in ngram_range:
    clf_pipeline = Pipeline(
            [('vectorizer', CountVectorizer(ngram_range=n)),
            ('regression', LogisticRegression())])
    score = cross_val_score(clf_pipeline, sms_text, labels, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', n)
    print('Cross-val-score f1: %.4f\n' % score)

Ngram_range:  (2, 2)
Cross-val-score f1: 0.8168

Ngram_range:  (3, 3)
Cross-val-score f1: 0.7250

Ngram_range:  (1, 3)
Cross-val-score f1: 0.9223



Повторим аналогичный эксперимент, используя вместо логистической регрессии MultinomialNB().

In [91]:
scores = []
for ngram in ngram_range:
    X_counts = CountVectorizer(ngram_range=ngram).fit_transform(sms_text)
    bayes = MultinomialNB()
    score = cross_val_score(bayes, X_counts, labels, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)

Ngram_range:  (2, 2)
Cross-val-score f1: 0.6451

Ngram_range:  (3, 3)
Cross-val-score f1: 0.3786

Ngram_range:  (1, 3)
Cross-val-score f1: 0.8878



Попробуем использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах.

In [94]:
clf_pipeline = Pipeline(
            [('vectorizer', TfidfVectorizer()),
            ('regression', LogisticRegression())])
score = cross_val_score(clf_pipeline, sms_text, labels, scoring='f1', cv=10).mean()
score

0.878423460536647

Как мы видим, качество на кросс-валидации по сравнению с CountVectorizer на униграммах заметно понизилось.